In [19]:
import pandas as pd

data = {
    "CLIENT": ["AXPO1", "AXPO2", "AXPO3", "AXPO4", "AXPO5"],
    "START-DATE": ["01 Jan 2023", "01 Jan 2023", "01 Jan 2023", "01 Jan 2023", "01 Jan 2023"],
    "END-DATE": ["31 Dec 2023", "31 Dec 2023", "31 Dec 2023", "31 Dec 2023", "31 Dec 2023"],
    "REGION": ["Flanders", "Walloon", "Walloon", "Walloon", "Flanders"],
    "REDUCTION": ["Yes", "No", "Yes", "Yes", "No"],
    "WAL. CERT.": [None, 70.00, 70.00, 70.00, None],
    "FLA. CERT.": [99.00, None, None, None, 99.00],
    "WKK CERT.": [29.00, None, None, None, 29.00],
    "2023-Q1": [78650, 210, 14980, 30880, 1030],
    "2023-Q2": [59750, 200, 18880, 35700, 3530],
    "2023-Q3": [43450, 200, 19230, 37820, 3260],
    "2023-Q4": [73150, 200, 18190, 33680, 1500]
}

df = pd.DataFrame(data)

# Quotas
quotas_walloon = {
    "without_reduction": 39.80,
    "with_reduction": [29.85, 19.90, 5.97, 3.98]
}

quotas_flanders = {
    "without_reduction_flemish": 18.00,
    "without_reduction_wkk": 11.20,
    "with_reduction_flemish": [18.00, 9.54, 3.60, 3.60, 0.36],
    "with_reduction_wkk": [11.20, 5.94, 5.60, 2.24, 1.68]
}

# Calculate total certificates and costs
results = []

for index, row in df.iterrows():
    total_certificates = 0
    total_cost = 0
    
    if row["REGION"] == "Walloon":
        for quarter in ["2023-Q1", "2023-Q2", "2023-Q3", "2023-Q4"]:
            consumption = row[quarter]
            
            if row["REDUCTION"] == "No":
                # Flat rate for clients without reduction
                certificates = consumption * (quotas_walloon["without_reduction"] / 100)
            else:
                # Apply the respective quota to the total consumption for clients with reduction
                quota = quotas_walloon["with_reduction"]
                tranches = [5000, 20000, 50000, float('inf')]
                remaining_consumption = consumption
                certificates = 0
                for i, tranche in enumerate(tranches):
                    if remaining_consumption > tranche:
                        applicable_consumption = tranche
                    else:
                        applicable_consumption = remaining_consumption

                    certificates += applicable_consumption * (quota[i] / 100)
                    remaining_consumption -= applicable_consumption

                    if remaining_consumption <= 0:
                        break

            total_certificates += certificates
        
        total_cost = total_certificates * row["WAL. CERT."]
        
    elif row["REGION"] == "Flanders":
        total_consumption = row["2023-Q1"] + row["2023-Q2"] + row["2023-Q3"] + row["2023-Q4"]
        
        if row["REDUCTION"] == "No":
            # Flat rates for clients without reduction
            flemish_certificates = total_consumption * (quotas_flanders["without_reduction_flemish"] / 100)
            wkk_certificates = total_consumption * (quotas_flanders["without_reduction_wkk"] / 100)
        else:
            # Apply the respective quota to the total consumption for clients with reduction
            flemish_certificates = 0
            wkk_certificates = 0
            remaining_consumption = total_consumption
            tranches = [1000, 19000, 80000, 150000, float('inf')]
            for i, tranche in enumerate(tranches):
                if remaining_consumption > tranche:
                    applicable_consumption = tranche
                else:
                    applicable_consumption = remaining_consumption

                flemish_certificates += applicable_consumption * (quotas_flanders["with_reduction_flemish"][i] / 100)
                wkk_certificates += applicable_consumption * (quotas_flanders["with_reduction_wkk"][i] / 100)
                remaining_consumption -= applicable_consumption
                
                if remaining_consumption <= 0:
                    break
        
        total_certificates = flemish_certificates + wkk_certificates
        total_cost = (flemish_certificates * row["FLA. CERT."]) + (wkk_certificates * row["WKK CERT."])
    
    results.append({
        "CLIENT": row["CLIENT"],
        "TOTAL_CERTIFICATES": total_certificates,
        "TOTAL_COST": total_cost
    })


results_df = pd.DataFrame(results)
results_df


,CLIENT,TOTAL_CERTIFICATES,TOTAL_COST
0,AXPO1,19455.200,1284542.80
1,AXPO2,322.380,22566.60
2,AXPO3,16174.720,1132230.40
3,AXPO4,24163.376,1691436.32
4,AXPO5,2721.440,196353.76
